In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from sklearn import manifold

In [1]:
import torch
import sys
import os
import importlib
import torch
import numpy
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict
import pickle
import random
# from asnorm import *

In [4]:
# make cohort set

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [3]:
sys.path.append('/workspace/GREAT_ASV_system/train_dist')

In [7]:
# from DatasetLoader import loadWAV

In [4]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNN').__getattribute__('MainModel')

In [5]:
# EPACA-TDNN
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNN.py, Embedding size is 192, Channels 1024, Spec_aug False.


In [6]:
model_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/ECAPA-1024+DAT+(sdsv/mozila)/model/model000000056.model'

In [19]:
# train_list = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_pretrain_10960.txt'
train_list = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_DA_10960.txt'
train_list_FA = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_normal_FA1417.txt'
train_list_EN = '/workspace/DATASET/server9_ssd/sdsv21/sdsv21_normal_EN9543.txt'
train_path = '/workspace/DATASET/server9_ssd/sdsv21'
test_list = '/workspace/DATASET/server9_ssd/sdsv21/task2_devtriallist.txt'
enroll_list = '/workspace/DATASET/server9_ssd/sdsv21/sdsv20_enrolllist.txt'
test_path = '/workspace/DATASET/server9_ssd/sdsv21'
# score_file = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/'
result_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/031401'
# out_path_1 = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/'

In [20]:
if not os.path.isdir(result_path):
    os.makedirs(result_path)

In [21]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [22]:
loaded_state = torch.load(model_path, map_location="cuda:0")

In [23]:
# self_state = self.__model__.module.state_dict()
# loaded_state = torch.load(path, map_location="cuda:%d"%self.gpu)
# loaded_state = torch.load(path, map_location="cpu")
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name

    ## pass spk clf weight
    if '__L__' in name:
        print('pass __L__ classerfier W')
        continue

    ## pass DA weight
    if 'DA_module' in name:
        print('pass DA_module params:'+name)
        continue

    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

pass __L__ classerfier W
pass DA_module params:DA_module.backbone.torchfb.0.flipped_filter
pass DA_module params:DA_module.backbone.torchfb.1.spectrogram.window
pass DA_module params:DA_module.backbone.torchfb.1.mel_scale.fb
pass DA_module params:DA_module.backbone.layer1.conv.weight
pass DA_module params:DA_module.backbone.layer1.bn.weight
pass DA_module params:DA_module.backbone.layer1.bn.bias
pass DA_module params:DA_module.backbone.layer1.bn.running_mean
pass DA_module params:DA_module.backbone.layer1.bn.running_var
pass DA_module params:DA_module.backbone.layer1.bn.num_batches_tracked
pass DA_module params:DA_module.backbone.layer2.0.conv.weight
pass DA_module params:DA_module.backbone.layer2.0.bn.weight
pass DA_module params:DA_module.backbone.layer2.0.bn.bias
pass DA_module params:DA_module.backbone.layer2.0.bn.running_mean
pass DA_module params:DA_module.backbone.layer2.0.bn.running_var
pass DA_module params:DA_module.backbone.layer2.0.bn.num_batches_tracked
pass DA_module para

In [25]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [26]:
def loadWAV(filename, max_frames):

    # Maximum audio length
    max_audio = max_frames * 160 + 240

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)

    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]

    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [27]:
loaded_state['model']['__L__.W'].size()

torch.Size([192, 10960])

In [28]:
W_matrix = loaded_state['model']['__L__.W']

In [29]:
W_matrix = W_matrix.t()

In [30]:
W_matrix.size()

torch.Size([10960, 192])

In [31]:
with open(os.path.join(train_path, train_list)) as dataset_file:
    lines = dataset_file.readlines()

dictkeys = list(set([x.split()[0] for x in lines]))
dictkeys.sort()
dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
len_dictkeys_ori = len(dictkeys)
if True:
    len_dictkeys = len(dictkeys)
    for ii, key in enumerate(list(dictkeys.keys())):
        dictkeys[key+'_slow'] = ii + len_dictkeys
        dictkeys[key+'_fast'] = ii + len_dictkeys*2

    assert len(dictkeys) == 3*len_dictkeys_ori

In [32]:
list(dictkeys.keys())[-1]

'spk_001587_fast'

In [33]:
list(dictkeys.keys())[10960-1]

'spk_001587'

In [34]:
with open(os.path.join(train_path, train_list_FA)) as dataset_file:
    FA_lines = dataset_file.readlines()
FA_dictkeys = list(set([x.split()[0] for x in FA_lines]))

In [35]:
len(FA_dictkeys)

1417

In [36]:
FA_list = []
for i in FA_dictkeys:
    print(dictkeys[i])
    FA_list.append(dictkeys[i])

10375
606
223
10660
659
153
10719
10620
426
221
10404
10702
227
87
620
480
80
77
10466
10781
10721
495
10525
10495
10872
10629
86
686
179
467
10909
636
10639
10604
548
492
592
488
596
108
95
324
586
10928
284
122
130
10806
10583
497
473
230
10482
10457
652
436
10953
594
317
816
10696
10770
616
510
10671
357
10464
10882
625
685
444
10560
10804
10400
10384
10788
10865
185
265
181
339
576
78
10507
246
10432
10379
10449
53
169
10574
10791
2
653
129
10642
10643
10584
535
248
458
517
10711
748
49
10434
787
599
356
10745
10635
10442
504
10729
506
59
10392
661
10930
10430
10637
10794
351
10447
668
809
733
10661
432
482
10672
10740
10679
137
611
18
708
10541
197
10866
749
239
391
10581
10819
255
463
27
10828
10388
76
151
601
45
106
51
10790
10477
10553
10764
10372
4
103
655
10840
501
218
720
394
505
10600
464
90
10827
10923
283
196
150
71
67
10416
776
215
238
10483
797
10489
420
0
111
430
273
406
10820
10493
165
672
295
10571
10425
421
477
10594
41
318
448
234
10575
637
538
91
142
10554
520
104

In [37]:
FA_list.sort()

In [38]:
FA_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [39]:
10959-10372+1

588

In [40]:
828-0+1

829

In [41]:
with open(os.path.join(train_path, train_list_EN)) as dataset_file:
    EN_lines = dataset_file.readlines()
EN_dictkeys = list(set([x.split()[0] for x in EN_lines]))

In [42]:
len(EN_dictkeys)

9543

In [43]:
EN_dictkeys[-5]

'id06025'

In [44]:
EN_list = []
for i in EN_dictkeys:
    print(dictkeys[i])
    EN_list.append(dictkeys[i])

4703
8573
7995
1757
6187
2351
7975
9488
2475
1104
8678
1997
8264
10042
1233
4815
4963
5985
9005
10057
3722
1048
7931
10323
1524
7930
5838
2529
8471
5277
1045
7010
1171
2260
7310
993
1845
1124
10222
7305
8278
7366
902
1919
3046
4403
6748
6955
1335
3322
4884
2053
6119
9947
7863
2908
9778
4855
6769
9274
7116
3177
1416
5891
7729
4763
2464
10321
9556
2879
8944
9404
4144
2563
8855
8739
8821
887
3172
7168
8663
6567
9480
8626
8977
4680
2705
8606
971
3429
8079
3423
9647
7170
4357
3432
6301
7525
8078
2988
4273
6280
9013
2535
2393
2154
6523
7013
8783
3207
6156
5067
1698
1773
7177
3831
3944
6897
5207
6260
9228
1129
1770
9448
4339
3379
4817
10080
3840
6366
9293
10128
5336
6221
6556
6715
7632
2706
4626
5007
8715
4412
5702
7204
8352
2907
4925
2716
3092
1852
5106
1213
9168
10082
4467
4491
4283
5737
1902
5401
7814
2408
1482
4490
984
5513
4813
7260
10215
7766
7595
6251
5950
9405
7203
871
1369
5706
5045
5803
2881
995
7709
949
8338
6840
8293
7364
5136
5971
4300
5762
5920
9586
2405
2105
8481
7886
5239
1349

2710
9570
2497
4177
2192
4331
10308
1671
10310
7686
1864
6100
3936
6239
9376
1136
8754
1619
914
3748
4145
2513
2890
6289
2151
6513
5275
6680
3763
5593
5774
4935
3461
6217
8779
4325
4551
8635
6343
6378
2092
4762
2633
8360
4395
8467
970
1909
5309
9717
5879
5575
6161
1890
10342
2587
3553
10245
9940
2676
3449
6914
5567
7207
1100
9084
8068
7336
2266
5183
6269
1661
2837
957
6040
4543
3238
9032
4911
2767
2198
1147
9621
9031
1696
1409
7322
6308
7333
1098
5848
6741
7410
3949
4912
8105
8632
2447
5816
6609
7443
5721
3511
3473
8528
2870
2648
6595
5076
3834
2739
5031
6543
7444
8096
9350
1322
10163
3031
1949
4747
882
1217
5967
10277
5465
2694
8147
9188
6963
4288
5286
6108
7839
2623
6364
4248
9052
5289
8872
6635
5563
9151
10227
3113
5874
9684
1093
6039
3555
2367
3154
5281
5817
3175
6771
1498
9907
870
7430
3014
4492
4307
6346
6134
8500
5758
6503
3736
8032
5391
2546
9297
9838
3217
7342
2657
7851
3044
4937
4571
9923
7415
1808
3111
8312
7566
7259
9636
8544
8643
2287
2915
3965
7209
5190
7128
9176
8697
963

9835
7155
5319
2018
9523
8782
9610
6960
4516
10083
2687
3864
4531
6786
1178
2347
2162
10197
6131
2387
9600
1907
7092
7316
9639
8060
3686
10201
1598
1002
5725
3339
1400
6325
9880
5943
2042
2161
8795
6599
3469
8416
1180
8574
8199
1167
8620
7572
861
1341
8125
10070
5677
9321
1930
904
2417
8841
6455
2905
4783
5834
8932
1206
6354
4016
9411
3451
3431
922
9275
2547
2378
3472
8233
3095
6210
8102
1328
2568
8372
7049
3862
4575
1183
1204
7890
1995
3973
4189
979
1558
893
6584
1052
6652
7476
3991
4362
2259
9856
9342
2471
8686
1305
8246
3032
6891
10137
3904
4265
5226
8104
6942
1504
9571
2394
8420
4560
9828
8213
9020
9253
3585
1134
1403
3407
4795
6220
7306
7663
1912
3101
8171
1565
1319
6329
3079
6570
3352
9137
2699
7002
8308
3335
4969
6316
10168
1298
2279
3952
5150
4832
5022
1472
4556
6337
3466
9390
8434
7708
9760
9651
2028
9476
4724
4166
1412
5175
5054
5595
6138
5676
4378
9069
7388
7156
9579
4184
9630
4426
8208
4430
1299
4957
6938
7277
7993
10056
5798
6468
923
4440
1411
8790
2916
3061
9441
6137
4301

8629
1424
7153
3685
3363
2058
4851
10030
3564
5821
8655
9161
4572
5746
8884
6077
4580
5638
8554
9987
4155
5356
1115
5051
9124
5945
4173
10069
2434
7985
1005
3311
3690
10327
6348
7479
4948
2488
3138
9551
2817
6350
6020
9837
2396
9123
1513
5576
6863
7073
8400
10120
3734
4210
6183
1477
6123
7416
1225
9757
1200
3360
5507
9497
10129
9381
7971
4408
1009
1616
7172
2618
8163
5077
5585
6104
4809
2276
3781
5354
4268
1439
1174
8012
3124
4728
7312
7786
7780
9734
3902
2322
6592
1932
5129
1588
6140
3272
6376
3167
7921
3446
4559
9894
4152
5521
9384
5262
9236
5340
9014
8952
3021
4245
2147
2647
10264
1076
5185
9459
5905
5443
4769
8269
1350
4779
2433
5781
9721
8801
2641
3338
1704
4864
2382
1737
2484
4921
7544
8619
8640
2212
6856
924
9795
9567
6882
7871
4715
4782
1373
2875
3218
7200
5122
4176
2963
9817
4848
968
2823
2062
4392
9614
10164
1789
1108
5835
8090
8337
7736
6508
1677
4712
3162
10300
3752
8842
9442
6907
7668
3853
10003
3089
5317
9290
9562
926
9793
1336
6728
9085
3479
4856
1753
2247
8162
3382
2831

In [45]:
EN_list.sort()

In [46]:
EN_list

[829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 855,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 871,
 872,
 873,
 874,
 875,
 876,
 877,
 878,
 879,
 880,
 881,
 882,
 883,
 884,
 885,
 886,
 887,
 888,
 889,
 890,
 891,
 892,
 893,
 894,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995

In [47]:
cohort_FA_spk_dict_mean = {}
cohort_EN_spk_dict_mean = {}

In [48]:
for count, label_num in enumerate(FA_list):
    cohort_FA_spk_dict_mean[label_num] = F.normalize(W_matrix[label_num], p=2, dim=0).detach().cpu()
print(count)

1416


In [49]:
for count, label_num in enumerate(EN_list):
    cohort_EN_spk_dict_mean[label_num] = F.normalize(W_matrix[label_num], p=2, dim=0).detach().cpu()
print(count)

9542


In [50]:
cohort_FA_spk_dict_mean_nm = cohort_FA_spk_dict_mean
cohort_EN_spk_dict_mean_nm = cohort_EN_spk_dict_mean

In [51]:
1417 + 9543

10960

In [99]:
# plot embs

In [52]:
out_list = numpy.zeros([0, 192])
for i in cohort_FA_spk_dict_mean:
    out_list = numpy.append(out_list, cohort_FA_spk_dict_mean[i][None, :].detach().cpu().numpy(), axis=0)

out_list_2 = numpy.zeros([0, 192])
for i in cohort_EN_spk_dict_mean:
    out_list_2 = numpy.append(out_list_2, cohort_EN_spk_dict_mean[i][None, :].detach().cpu().numpy(), axis=0)


out_label = len(out_list)*[0] +2000*[1]
out_list_f = numpy.append(out_list, out_list_2[:2000], axis=0)

In [53]:
out_list_f.shape

(3417, 192)

In [54]:
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
Y = tsne.fit_transform(out_list_f)

NameError: name 'manifold' is not defined

In [ ]:
fig = plt.figure()
Y1 = Y[0:out_list.shape[0]]
out_label_1 = out_label[0:out_list.shape[0]]
Y2 = Y[out_list.shape[0]:]
out_label_2 = out_label[out_list.shape[0]:]
a = plt.scatter(Y1[:, 0], Y1[:, 1], color='b', s=1)
b = plt.scatter(Y2[:, 0], Y2[:, 1], color='r', s=1)
plt.legend([a, b],['FA', 'EN'], scatterpoints=5)
plt.title('protos')

In [ ]:
# plot and find alpha

In [56]:
ORI_TASK2 = '/workspace/DATASET/server9_ssd/sdsv21/task2_dev_trials.txt'

In [57]:
with open(ORI_TASK2, 'r') as f:
    trial_info = f.readlines()

In [58]:
trial_info

['model_15007 evl_055907 imp FA m\n',
 'model_15007 evl_069246 imp FA m\n',
 'model_15008 evl_036778 imp FA m\n',
 'model_15008 evl_048439 imp FA m\n',
 'model_15011 evl_040546 imp FA f\n',
 'model_15014 evl_039053 imp EN m\n',
 'model_15014 evl_066055 tgt FA m\n',
 'model_15017 evl_056936 imp FA m\n',
 'model_15027 evl_019457 imp EN f\n',
 'model_15039 evl_057664 imp FA f\n',
 'model_15053 evl_004129 imp EN f\n',
 'model_15053 evl_047540 imp FA f\n',
 'model_15056 evl_032901 imp FA f\n',
 'model_15056 evl_043718 imp EN f\n',
 'model_15058 evl_051163 imp EN m\n',
 'model_15061 evl_008734 imp FA f\n',
 'model_15063 evl_007791 imp FA f\n',
 'model_15063 evl_023293 tgt FA f\n',
 'model_15063 evl_028774 imp FA f\n',
 'model_15067 evl_020615 imp FA m\n',
 'model_15067 evl_043130 imp FA m\n',
 'model_15067 evl_066772 imp FA m\n',
 'model_15085 evl_005092 tgt FA f\n',
 'model_15101 evl_007382 imp EN m\n',
 'model_15101 evl_025570 tgt FA m\n',
 'model_15101 evl_032840 imp FA m\n',
 'model_1510

In [60]:
# imp_FA_idx = []
# imp_EN_idx = []
# for count, i in enumerate(trial_info):
#     data = i.split(' ')
#     if data[2] == 'imp':
#         if data[3] == 'FA':
#             imp_FA_idx.append(count)
#         elif data[3] == 'EN':
#             imp_EN_idx.append(count)

In [59]:
imp_FA = []
imp_EN = []
for count, i in enumerate(trial_info):
    data = i.split(' ')
    if data[2] == 'imp':
        if data[3] == 'FA':
            imp_FA.append(data[0]+' '+data[1])
        elif data[3] == 'EN':
            imp_EN.append(data[0]+' '+data[1])

In [61]:
len(imp_EN)

2557

In [62]:
len(imp_FA)

3046

In [104]:
# enroll sdsv21

In [63]:
enrollfilename = enroll_list
enroll_files = {}
## Read all enroll lines
with open(enrollfilename) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Enroll file should have line length >= 2
        assert len(data) >= 2

        enroll_files[data[0]] = data[1:]

In [64]:
len(enroll_files)

15555

In [67]:
enroll_dict = {}

In [68]:
for count, enroll_id in enumerate(enroll_files):
    for file in enroll_files[enroll_id]:
        wavline = file
        wavline = os.path.join(test_path, wavline)
        raw_inp = loadWAV(wavline, max_frames=0)
        raw_inp = torch.FloatTensor(raw_inp).cuda()

        ref_feat = S.forward(raw_inp).detach().cpu()

        if enroll_id not in enroll_dict.keys():
            enroll_dict[enroll_id] = ref_feat
        else:
            enroll_dict[enroll_id] = torch.cat([enroll_dict[enroll_id], ref_feat], axis=0)

    print(count+1, end='\r')


In [69]:
enroll_dict_nm = {}
for i in enroll_dict:
    enroll_dict_nm[i] = F.normalize(torch.mean(enroll_dict[i], dim=0).squeeze(0), p=2, dim=0)

In [70]:
len(enroll_dict_nm)

15555

In [111]:
# test sdsv21

In [71]:
listfilename = test_list
files = []
with open(listfilename) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split()

        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [72]:
len(setfiles)

6594

In [73]:
test_dict = {}

In [74]:
for count, line in enumerate(setfiles):
    wavline = line
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline, max_frames=0)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu()

    test_dict[line] = ref_feat
    
    if ((count+1) % 1000) == 0:
        print((count+1)//1000, end='\r')

In [75]:
len(test_dict)

6594

In [76]:
test_dict_nm = {}
for i in test_dict:
    test_dict_nm[i] = F.normalize(test_dict[i].squeeze(0), p=2, dim=0)

In [80]:
test_dict.keys()

dict_keys(['task2_evaluation/evl_000004.wav', 'task2_evaluation/evl_000019.wav', 'task2_evaluation/evl_000022.wav', 'task2_evaluation/evl_000024.wav', 'task2_evaluation/evl_000043.wav', 'task2_evaluation/evl_000058.wav', 'task2_evaluation/evl_000072.wav', 'task2_evaluation/evl_000073.wav', 'task2_evaluation/evl_000081.wav', 'task2_evaluation/evl_000097.wav', 'task2_evaluation/evl_000101.wav', 'task2_evaluation/evl_000107.wav', 'task2_evaluation/evl_000122.wav', 'task2_evaluation/evl_000126.wav', 'task2_evaluation/evl_000152.wav', 'task2_evaluation/evl_000159.wav', 'task2_evaluation/evl_000162.wav', 'task2_evaluation/evl_000165.wav', 'task2_evaluation/evl_000172.wav', 'task2_evaluation/evl_000191.wav', 'task2_evaluation/evl_000196.wav', 'task2_evaluation/evl_000198.wav', 'task2_evaluation/evl_000205.wav', 'task2_evaluation/evl_000209.wav', 'task2_evaluation/evl_000225.wav', 'task2_evaluation/evl_000231.wav', 'task2_evaluation/evl_000240.wav', 'task2_evaluation/evl_000241.wav', 'task2_ev

In [82]:
imp_FA_s = []
imp_EN_s = []

for i in imp_FA:
    data = i.split(' ')
    e, t = data[0], data[1]
    t = 'task2_evaluation/'+t+'.wav'
    imp_FA_s.append(F.cosine_similarity(enroll_dict_nm[e], test_dict_nm[t], dim=0).numpy())
    
for i in imp_EN:
    data = i.split(' ')
    e, t = data[0], data[1]
    t = 'task2_evaluation/'+t+'.wav'
    imp_EN_s.append(F.cosine_similarity(enroll_dict_nm[e], test_dict_nm[t], dim=0).numpy())

imp_FA_s = numpy.array(imp_FA_s)
imp_EN_s = numpy.array(imp_EN_s)

In [83]:
numpy.mean(imp_FA_s)

0.05037739

In [84]:
numpy.mean(imp_EN_s)

0.04556346

In [85]:
cohort_FA_spk_dict_mean_nm[]

1417

In [116]:
count = 0
imp_FA2cohort_s = []
for i in cohort_FA_spk_dict_mean_nm:
    if i < 829:
        continue
    cohort_emb = cohort_FA_spk_dict_mean_nm[i]
    for j in imp_FA:
        j = 'task2_evaluation/'+j.split(' ')[1]+'.wav'
        
        test_emb = test_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, test_emb, dim=0).numpy().astype(numpy.float16)
        imp_FA2cohort_s.append(score)

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

1791048


In [117]:
imp_FA2cohort_s = numpy.array(imp_FA2cohort_s)

In [118]:
count = 0
imp_EN2cohort_s = []
for i in cohort_FA_spk_dict_mean_nm:
    if i < 829:
        continue
    cohort_emb = cohort_FA_spk_dict_mean_nm[i]
    for j in imp_EN:
        j = 'task2_evaluation/'+j.split(' ')[1]+'.wav'
        
        test_emb = test_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, test_emb, dim=0).numpy().astype(numpy.float16)
        imp_EN2cohort_s.append(score)

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

1503516


In [119]:
imp_EN2cohort_s = numpy.array(imp_EN2cohort_s)

In [120]:
numpy.mean(imp_FA2cohort_s) # sdsv

0.01694

In [121]:
numpy.mean(imp_EN2cohort_s) # sdsv

0.01509

In [98]:
numpy.mean(imp_FA2cohort_s) # mozila and sdsv

-0.000784

In [99]:
numpy.mean(imp_EN2cohort_s) # mozila and sdsv

-0.001986

In [92]:
numpy.mean(imp_FA2cohort_s) # mozila and sdsv

-0.000784

In [93]:
numpy.mean(imp_EN2cohort_s) # mozila and sdsv

-0.001986

In [ ]:
# asnorm

In [119]:
15555 * 1417 // 100000

220

In [120]:
69542 * 1417 // 100000

985

In [121]:
count = 0
cohort2enroll = {}
for i in cohort_FA_spk_dict_mean_nm:
    cohort_emb = cohort_FA_spk_dict_mean_nm[i]
    for j in enroll_dict_nm:
        enroll_emb = enroll_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, enroll_emb, dim=0).numpy().astype(numpy.float16)
        cohort2enroll[str(i)+" "+j] = score

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

9146340


In [122]:
count = 0
cohort2test = {}
for i in cohort_FA_spk_dict_mean_nm:
    cohort_emb = cohort_FA_spk_dict_mean_nm[i]
    for j in test_dict_nm:
        test_emb = test_dict_nm[j]
        score = F.cosine_similarity(cohort_emb, test_emb, dim=0).numpy().astype(numpy.float16)
        cohort2test[str(i)+" "+j] = score

        count += 1
        if (count % 100000) == 0:
            print(count // 100000, end='\r')
                
print(count)

40890696


In [123]:
len(cohort2test)

40890696

In [124]:
cohort2enroll_save_path = os.path.join(result_path, 'tmp_cohort2enroll.txt')
cohort2test_save_path = os.path.join(result_path, 'tmp_cohort2test.txt')

In [125]:
with open(cohort2enroll_save_path, 'w') as f:
    for count, i in enumerate(cohort2enroll):
        line = '%s %.4f\n'%(i, cohort2enroll[i])
        f.write(line)
        if (count % 100000) == 0:
            print(count // 100000, end='\r')

In [126]:
with open(cohort2test_save_path, 'w') as f:
    for count, i in enumerate(cohort2test):
        line = '%s %.4f\n'%(i, cohort2test[i])
        f.write(line)
        if (count % 100000) == 0:
            print(count // 100000, end='\r')

In [127]:
cohort2enroll = None
cohort2test = None

In [128]:
## optional
# import pickle
# with open('tmp_chort_dict', 'wb') as f:
#     pickle.dump(cohort2test, f)
# import pickle
# with open('tmp_test_dict', 'wb') as f:
#     pickle.dump(test_dict, f)

In [129]:
import torch
import sys
import os
import torch
import numpy
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict

def as_norm_1(file_score_path, out_path, cohort2enroll_path, cohort2test_path, top_num=1000, hold_name=True, dict_sep=' '):
    # operation: norm_score = 0.5* ((score-mean_e_e)/str_e_e+(score-mean_t_t)/str_t_t)
    print('Score norm operation start, method: Adaptive_score_norm_type_1')
    print('')
    
    file_scores_as_norm_path = out_path
    enroll_dict = defaultdict(list)
    test_dict = defaultdict(list)

    with open(cohort2enroll_path, 'r') as f:
        for count, i in enumerate(f):
            cohort_utt, enroll_utt, score = i.strip().split(dict_sep)
            score = float(score)
            enroll_dict[enroll_utt].append([cohort_utt, score])
            if ((count+1) % 100000) == 0:
                print('read c2e:', (count+1)//100000, end='\r')
    
    with open(cohort2test_path, 'r') as f:
        for count, i in enumerate(f):
            cohort_utt, test_utt, score = i.strip().split(dict_sep)
            score = float(score)
            test_dict[test_utt].append([cohort_utt, score])
            if ((count+1) % 100000) == 0:
                print('read c2t:', (count+1)//100000, end='\r')
                
        
    mean_e_e = {}
    str_e_e = {}
    mean_t_t = {}
    str_t_t = {}
    
    print('')
    print('calculate enroll statistics')
    print('')
    
    for count, key in enumerate(enroll_dict):
        enroll_dict[key] = sorted(enroll_dict[key],key = lambda x:x[1], reverse = True)
        tmp_score_list = []
        for i in range(top_num):
            tmp_score_list.append(enroll_dict[key][i][1])
        mean_e_e[key] = numpy.mean(tmp_score_list)
        str_e_e[key] = numpy.std(tmp_score_list, ddof=1)
        if ((count+1) % 1000) == 0:
            print('cal e:', (count+1)//1000, end='\r')

    for count, key in enumerate(test_dict):
        test_dict[key] = sorted(test_dict[key],key = lambda x:x[1], reverse = True)
        tmp_score_list = []
        for i in range(top_num):
            tmp_score_list.append(test_dict[key][i][1])
        mean_t_t[key] = numpy.mean(tmp_score_list)
        str_t_t[key] = numpy.std(tmp_score_list, ddof=1)
        if ((count+1) % 1000) == 0:
            print('cal t:', (count+1)//1000, end='\r')

    print('')
    print('Scoring...')
    print('')
    
    file_scores = open(file_score_path)
    with open(file_scores_as_norm_path,'w') as f:
        for count, line in enumerate(file_scores):
            enroll_utt = line.split(' ')[1].strip()
            test_utt = line.split(' ')[2].strip()
            score = float(line.split(' ')[0].strip())
            norm_score = 0.5 * ((score-mean_e_e[enroll_utt])/str_e_e[enroll_utt]+(score-mean_t_t[test_utt])/str_t_t[test_utt])
            if hold_name:
                f.write('%.4f %s %s\n'%(norm_score, enroll_utt, test_utt))
            else:
                f.write('%.4f\n'%(norm_score))
            
            if ((count+1) % 10000) == 0:
                print((count+1)//10000, end='\r')
                
    file_scores.close()
    print('')
    print('Adaptive_score_norm_type_1 is finished')

In [134]:
as_norm_1(score_file, out_path_1, cohort2enroll_save_path, cohort2test_save_path, top_num=100, hold_name=True)

Score norm operation start, method: Adaptive_score_norm_type_1

read c2t: 40815 26 244
calculate enroll statistics

cal t: 69
Scoring...

461
Adaptive_score_norm_type_1 is finished


In [131]:
# as_norm_2(score_file, out_path_2, cohort2test, cohort2test, top_num=1000, hold_name=True)

In [132]:
# get asnormed score_file & scoring